# Introdution and Background

# Data Description

# Data Cleaning/Pre-processing

In [1]:
import pandas as pd
import numpy as np

## Income

In [2]:
income = pd.read_csv('Income_Data/cities_median_income.csv')

In [3]:
income.head()

,City,Median Income 2016,Median Income 2015,Median Income 2014,Median Income 2013,Median Income 2012,Median Income 2011,Median Income 2010
0,"Birmingham, AL","46,346","44,463","44,447","44,716","44,710","45,423","45,198"
1,"Montgomery, AL","63,847","62,201","62,568","62,418","61,896","62,645","61,229"
2,"Anchorage, AK","102,500","99,630","98,317","96,935","95,632","94,073","91,251"
3,"Juneau, AK","104,005","102,355","97,588","96,025","95,433","93,994","89,373"
4,"Phoenix, AZ","69,216","66,345","65,770","65,199","65,968","66,709","66,996"


In [4]:
# rename columns
income.columns = ['city', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

In [5]:
# unpivot
income = pd.melt(income, id_vars=['city'], var_name='year', value_name='median income') \
           .sort_values(['city', 'year']) \
           .reset_index(drop=True)

In [6]:
income.head()

,city,year,median income
0,"Albany, NY",2010,"73,470"
1,"Albany, NY",2011,"75,821"
2,"Albany, NY",2012,"76,707"
3,"Albany, NY",2013,"77,687"
4,"Albany, NY",2014,"78,285"


## Unemployment

In [20]:
unemployment = pd.read_csv('Income_Data/cities_unemployment_rate.csv')

In [21]:
unemployment.head()

,City,Unemployment Rate 2016,Unemployment Rate 2015,Unemployment Rate 2014,Unemployment Rate 2013,Unemployment Rate 2012,Unemployment Rate 2011,Unemployment Rate 2010
0,"Birmingham, AL",11.8%,13.3%,14.5%,15.3%,14.5%,13.6%,12.9%
1,"Montgomery, AL",8.4%,8.8%,8.9%,8.9%,8.7%,8.6%,7.9%
2,"Anchorage, AK",6.0%,6.7%,6.9%,7.3%,7.3%,7.2%,7.4%
3,"Juneau, AK",4.9%,5.2%,5.6%,5.3%,5.1%,5.3%,5.8%
4,"Phoenix, AZ",7.7%,8.8%,9.9%,10.6%,9.7%,8.7%,7.4%


In [22]:
# rename columns
unemployment.columns = ['city', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

In [23]:
# unpivot
unemployment = pd.melt(unemployment, id_vars=['city'], var_name='year', value_name='unemployment rate') \
           .sort_values(['city', 'year']) \
           .reset_index(drop=True)

In [25]:
# convert percentages to decimals
unemployment['unemployment rate'] = unemployment['unemployment rate'] \
                                    .map(lambda rate: float(rate.rstrip('%'))/100)

In [26]:
unemployment.head()

,city,year,unemployment rate
0,"Albany, NY",2010,0.060
1,"Albany, NY",2011,0.065
2,"Albany, NY",2012,0.070
3,"Albany, NY",2013,0.073
4,"Albany, NY",2014,0.068


## Traffic

In [27]:
traffic = pd.read_csv('Income_Data/cities_mean_commute_time.csv')

In [28]:
traffic.head()

,City,Mean Commute Time 2016,Mean Commute Time 2015,Mean Commute Time 2014,Mean Commute Time 2013,Mean Commute Time 2012,Mean Commute Time 2011,Mean Commute Time 2010
0,"Birmingham, AL",21.4,21.4,21.6,21.7,21.8,21.7,22.0
1,"Montgomery, AL",20.0,19.3,19.5,19.3,19.5,19.5,19.7
2,"Anchorage, AK",19.1,19.5,19.4,19.4,19.0,18.6,18.2
3,"Juneau, AK",15.6,15.1,14.8,15.1,15.5,15.2,15.3
4,"Phoenix, AZ",25.1,24.8,24.5,24.4,24.4,24.7,24.8


In [29]:
# rename columns
traffic.columns = ['city', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

In [31]:
# unpivot
traffic = pd.melt(traffic, id_vars=['city'], var_name='year', value_name='mean commute time') \
           .sort_values(['city', 'year']) \
           .reset_index(drop=True)

In [32]:
traffic.head()

,city,year,mean commute time
0,"Albany, NY",2010,19.7
1,"Albany, NY",2011,19.9
2,"Albany, NY",2012,20.0
3,"Albany, NY",2013,20.0
4,"Albany, NY",2014,20.1


## Crime

## Happiness

# Data Visualization

# Data Anaylsis and Results

# Privacy/Ethics Considerations

# Conclusions and Discussion